In [1]:
%pip install --upgrade pip 
%pip install pymongo
%pip install faker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from uuid import uuid4
from datetime import datetime
from contextlib import closing
from time import time
from random import choice as random_choice

from pymongo import MongoClient

from faker import Faker

In [5]:
HOST = 'localhost'
PORT = 27019

def get_mongo_client():
    return MongoClient(host=HOST,
                       port=PORT,
                       uuidRepresentation='standard')

In [ ]:
# create review collection
with closing(get_mongo_client()) as client:
   database = client[DB]

   database.drop_collection('review')

   review_schema = {
         "bsonType": "object",
         "required": ["user_id", "movie_id", "score", "is_delete", "dt"],
         "properties": {
            "user_id": {
               "bsonType": "binData"
            },
            "movie_id": {
               "bsonType": "binData"
            },
            "score": {
               "bsonType": "int"
            },
            "text": {
               "bsonType": "string"
            },
            "is_delete": {
               "bsonType": "bool"
            },
            "dt": {
               "bsonType": "date"
            }
         }
      }

   review_collection = database.create_collection(
      'review',
      check_exists=False,
      validator= {
         '$jsonSchema': review_schema
      })

   review_collection.create_index("movie_id")

In [7]:
# create bookmark collection
with closing(get_mongo_client()) as client:
   database = client[DB]

   database.drop_collection('bookmark')

   bookmark_schema = {
         "bsonType": "object",
         "required": ["user_id", "movie_id", "is_delete", "dt"],
         "properties": {
            "user_id": {
               "bsonType": "binData"
            },
            "movie_id": {
               "bsonType": "binData"
            },
            "is_delete": {
               "bsonType": "bool"
            },
            "dt": {
               "bsonType": "date"
            }
         }
      }

   bookmark_collection = database.create_collection(
      'bookmark',
      check_exists=False,
      validator= {
         '$jsonSchema': bookmark_schema
      })

   bookmark_collection.create_index("user_id")

In [44]:
# create review_rating collection
with closing(get_mongo_client()) as client:
    database = client[DB]

    database.drop_collection('review_rating')

    review_rating_schema = {
        "bsonType": "object",
        "required": ["user_id", "review_id", "score", "is_delete", "dt"],
        "properties": {
            "user_id": {
                "bsonType": "binData"
            },
            "review_id": {
                "bsonType": "binData"
            },
            "score": {
                "bsonType": "int"
            },
            "is_delete": {
               "bsonType": "bool"
            },
            "dt": {
               "bsonType": "date"
            }
        }
    }

    review_rating_collection = database.create_collection(
        'review_rating',
        check_exists=False,
        validator= {
            '$jsonSchema': review_rating_schema,
            'score': {
                '$in': [-1, 1]
            }
        }
    )

    review_rating_collection.create_index("review_id")

In [10]:
# check collections & indexes
with closing(get_mongo_client()) as client:
    database = client[DB]

    print(database.list_collection_names())
    print(database['review'].index_information())
    print(database['bookmark'].index_information())
    print(database['review_rating'].index_information())

['review', 'review_rating', 'bookmark']
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'movie_id_1': {'v': 2, 'key': [('movie_id', 1)]}}
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'user_id_1': {'v': 2, 'key': [('user_id', 1)]}}
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'review_id_1': {'v': 2, 'key': [('review_id', 1)]}}


In [45]:
fake: Faker = Faker()

USERS_COUNT = 500
USERS_IDS = [uuid4() for _ in range(USERS_COUNT)]

MOVIES_COUNT = 100
MOVIES_IDS = [uuid4() for _ in range(MOVIES_COUNT)]

In [25]:
# insert review
with closing(get_mongo_client()) as client:
    collection = client[DB]['review']

    start_time: float = time()
    for user_id in USERS_IDS:
        for movie_id in MOVIES_IDS:
            new_review = {
                'user_id': uuid4(),
                'movie_id': uuid4(),
                'score': fake.random_int(min=0, max=10),
                'text': fake.text(),
                'is_delete': False,
                'dt': fake.date_time_between(start_date="-1y", end_date="now")
            }

            collection.insert_one(new_review)

    insertion_time: float = time() - start_time
    insertion_speed: int = (USERS_COUNT * MOVIES_COUNT) // insertion_time

    print(f' -- Insert {USERS_COUNT * MOVIES_COUNT} elements. Speed: {insertion_speed} records/sec')

 -- Insert 50000 elements. Speed: 296.0 records/sec


In [35]:
# insert bookmark
with closing(get_mongo_client()) as client:
    collection = client[DB]['bookmark']

    start_time: float = time()
    for user_id in USERS_IDS:
        for movie_id in MOVIES_IDS:
            new_bookmark = {
                'user_id': uuid4(),
                'movie_id': uuid4(),
                'is_delete': False,
                'dt': fake.date_time_between(start_date="-1y", end_date="now")
            }
            collection.insert_one(new_bookmark)

    insertion_time: float = time() - start_time
    insertion_speed: int = (USERS_COUNT * MOVIES_COUNT) // insertion_time

    print(f' -- Insert {USERS_COUNT * MOVIES_COUNT} bookmarks. Speed: {insertion_speed} records/sec')

 -- Insert 50000 bookmarks. Speed: 301.0 records/sec


In [46]:
# insert review rating
with closing(get_mongo_client()) as client:
    collection = client[DB]['review_rating']

    reviews_count_for_bench = 100
    users_count_for_bench = 500

    query_result = client[DB]['review'].find({}).limit(reviews_count_for_bench)
    all_reviews_ids = [review.get('id') for review in query_result]

    start_time: float = time()
    for user_id in USERS_IDS[:users_count_for_bench]:
        for review_id in all_reviews_ids:
            score = random_choice([-1, 1])
            new_review_rating = {
                'user_id': uuid4(),
                'review_id': uuid4(),
                'score': score,
                'is_delete': False,
                'dt': fake.date_time_between(start_date="-1y", end_date="now")
            }
            collection.insert_one(new_review_rating)

    insertion_time: float = time() - start_time
    insertion_speed: int = (reviews_count_for_bench * users_count_for_bench) // insertion_time

    print(f' -- Insert {reviews_count_for_bench * users_count_for_bench} elements. Speed: {insertion_speed} records/sec')

 -- Insert 50000 elements. Speed: 285.0 records/sec


In [30]:
# select review
with closing(get_mongo_client()) as client:
    collection = client[DB]['review']

    start_time = time()
    all_reviews = list(collection.find({}))
    selection_time = float(time() - start_time)

    reviews_len = len(all_reviews)
    selection_speed = reviews_len // selection_time

    print(f' -- Select {reviews_len} reviews. Speed: {selection_speed} records/sec')

 -- Select 100000 reviews. Speed: 73968.0 records/sec


In [41]:
# select bookmarks
with closing(get_mongo_client()) as client:
    collection = client[DB]['bookmark']

    start_time = time()
    all_bookmarks = list(collection.find({}))
    selection_time = float(time() - start_time)

    bookmarks_len = len(all_bookmarks)
    selection_speed: int = bookmarks_len // selection_time

    print(f' -- Select {bookmarks_len} bookmarks.Speed: {selection_speed} records/sec')

 -- Select 100000 bookmarks.Speed: 89049.0 records/sec


In [48]:
# select review ratings
with closing(get_mongo_client()) as client:
    collection = client[DB]['review_rating']

    start_time: float = time()
    all_review_ratings = list(collection.find({}))
    selection_time: float = float(time() - start_time)

    reviews_ratings_len = len(list(all_review_ratings))
    selection_speed: int = reviews_ratings_len // selection_time

    print(f' -- Select {reviews_ratings_len} reviews ratings. Speed: {selection_speed} records/sec')

 -- Select 50000 reviews ratings. Speed: 98695.0 records/sec


In [32]:
# update review
with closing(get_mongo_client()) as client:
    collection = client[DB]['review']

    all_reviews = list(collection.find({}).limit(50000))
    reviews_len = len(all_reviews)

    start_time: float = time()
    for review in all_reviews:
        filter = {
            'movie_id': review.get('movie_id'),
            'user_id': review.get('user_id'),
        }
        new_values = {
            'score': fake.random_int(min=0, max=10),
            'text': fake.text(),
            'dt': fake.date_time_between(start_date="-1y", end_date="now")
        }

        collection.update_one(filter, {'$set': new_values})

    updation_time: float = float(time() - start_time)
    updation_speed: int = reviews_len // updation_time

    print(f' -- Update {reviews_len} reviews. Speed: {updation_speed} records/sec')

 -- Update 50000 reviews. Speed: 282.606502125879 records/sec
